In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
cd /content/drive/My\ Drive

/content/drive/My Drive


# XG Boost Classifier

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
from xgboost import XGBClassifier

## Reading data

In [35]:
df = pd.read_csv("./Data/full_ohe.csv")
df.head()

age  default  ...  day_of_week_tue  day_of_week_wed
0  0.481481        1  ...                0                0
1  0.493827        0  ...                0                0
2  0.246914        1  ...                0                0
3  0.283951        1  ...                0                0
4  0.481481        1  ...                0                0

[5 rows x 56 columns]

In [36]:
# raw data
X = df.drop('y', axis=1).values
y = df['y'].values
X.shape,y.shape

((73668, 55), (73668,))

#### Test Train Split

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

## Training Classifier

In [38]:
clf = XGBClassifier()
clf.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

## Testing

In [0]:
predict = clf.predict(x_test)

In [0]:
acc = accuracy_score(predict, y_test)
pre = precision_score(predict, y_test)
rec = recall_score(predict, y_test)
f1 = f1_score(predict, y_test)

In [41]:
table = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
  'Score': [acc, pre, rec, f1]
    })
table

Metric     Score
0   Accuracy  0.750192
1  Precision  0.649490
2     Recall  0.814693
3   F1 Score  0.722772

In [42]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

probs = clf.predict_proba(x_test)
preds = probs[:,1]
fpr, tpr, threshold = roc_curve(y_test, preds)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

# Feature Importance

In [0]:
df = pd.read_csv("./Data/full_le.csv")
X = df.drop('y', axis=1).values
y = df['y'].values
pp=df.drop('y', axis=1)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
rfc = XGBClassifier()
rfc.fit(x_train, y_train)
feature_importances = pd.DataFrame(rfc.feature_importances_,index = pp.columns,columns=['importance']).sort_values('importance',ascending=False)

In [44]:
feature_importances

importance
nr.employed       0.642455
cons.conf.idx     0.053538
euribor3m         0.052748
pdays             0.049735
cons.price.idx    0.032667
default           0.026466
poutcome          0.024481
month             0.023509
contact           0.021803
day_of_week       0.015324
age               0.010040
campaign          0.008273
education         0.008165
previous          0.007178
emp.var.rate      0.006417
job               0.005009
marital           0.004482
housing           0.004118
loan              0.003591
pdays2            0.000000

### This is a list of all features ranked based on their importance to the random forest classifier

# New Section